In [7]:
import pandas as pd
from pathlib import Path
import re

PATH = Path().parent
__DATA__ = PATH / "db"

# Load the datasets
df_sheet1 = pd.read_excel(__DATA__ / 'hasil hasil belajar.xlsx', sheet_name=0)
df_sheet2 = pd.read_excel(__DATA__ / 'hasil hasil belajar.xlsx', sheet_name=1)
df_sheet3 = pd.read_excel(__DATA__ / 'hasil hasil belajar.xlsx', sheet_name=2)

# Rename columns
df_sheet1 = df_sheet1.rename(columns={'keyakinan': 'certainty_factor'})
df_sheet2 = df_sheet2.rename(columns={'keyakinan': 'certainty_factor'})
df_sheet3 = df_sheet3.rename(columns={'kepercayaan': 'certainty_factor'})

# Convert `lama_berlatih (min 5, mak 15)` to minutes as float
def convert_to_minutes(time_str):
    minutes, seconds = map(int, re.findall(r'\d+', time_str))
    return minutes + seconds / 60

df_sheet1['lama_berlatih (min 5, mak 15)'] = df_sheet1['lama_berlatih (min 5, mak 15)'].apply(convert_to_minutes)
df_sheet2['lama_berlatih (min 5, mak 15)'] = df_sheet2['lama_berlatih (min 5, mak 15)'].apply(convert_to_minutes)

# Drop unnecessary columns
df_sheet1 = df_sheet1.drop(['kesimpulannya', 'no'], axis=1)
df_sheet2 = df_sheet2.drop(['kesimpulannya', 'no'], axis=1)
df_sheet3 = df_sheet3.drop(['Unnamed: 0'], axis=1)

# Define the membership function
def calculate_membership(x, points):
    if x <= points[0] or x >= points[3]:
        return 0
    elif points[0] < x <= points[1]:
        return (x - points[0]) / (points[1] - points[0])
    elif points[1] < x <= points[2]:
        return 1
    elif points[2] < x < points[3]:
        return (points[3] - x) / (points[3] - points[2])

# Apply the function to calculate fuzzy values for the specified columns
for df in [df_sheet1, df_sheet2]:
    df['rata2 Lama akses materi'] = df['rata2 Lama akses materi'].apply(lambda x: calculate_membership(x, [300, 450, 550, 700]))
    df['lama_berlatih (min 5, mak 15)'] = df['lama_berlatih (min 5, mak 15)'].apply(lambda x: calculate_membership(x, [5, 10, 20, 25]))
    df['nilai'] = df['nilai'].apply(lambda x: calculate_membership(x, [0, 40, 70, 100]))

# Concatenate the datasets
df_combined = pd.concat([df_sheet1, df_sheet2], ignore_index=True)

# Define the fuzzy logic system function
def fuzzy_logic_system(rata2_lama_akses_materi, lama_berlatih, nilai):
    max_cf = -1
    result = None

    for _, row in df_sheet3.iterrows():
        cf_rata2_lama_akses_materi = calculate_membership(rata2_lama_akses_materi, [300, 450, 550, 700])
        cf_lama_berlatih = calculate_membership(lama_berlatih, [5, 10, 20, 25])
        cf_nilai = calculate_membership(nilai, [0, 40, 70, 100])

        cf = min(cf_rata2_lama_akses_materi, cf_lama_berlatih, cf_nilai)
        if cf > max_cf:
            max_cf = cf
            result = row['lulus']

    return result, max_cf

# Apply the function to the combined DataFrame
df_combined[['lulus', 'certainty_factor_lulus']] = df_combined.apply(
    lambda row: fuzzy_logic_system(row['rata2 Lama akses materi'], row['lama_berlatih (min 5, mak 15)'], row['nilai']),
    axis=1,
    result_type='expand'
)

# Display the first 5 rows of the combined DataFrame
print("Combined DataFrame with Fuzzy Logic Results:")
# print(df_combined.head().to_markdown(index=False, numalign="left", stralign="left"))
print(df_combined['lulus'].value_counts())

Combined DataFrame with Fuzzy Logic Results:
lulus
tidak lulus    107
Name: count, dtype: int64


In [13]:
df1 = pd.read_excel(__DATA__ / 'hasil hasil belajar.xlsx', sheet_name=0)
df2 = pd.read_excel(__DATA__ / 'hasil hasil belajar.xlsx', sheet_name=1)

concat = pd.concat([df1, df2], ignore_index=True)
#rata rata kolom 1
rata_rata = concat['rata2 Lama akses materi'].mean()
#minimum kolom 1
minimum = concat['rata2 Lama akses materi'].min()
#maksimum kolom 1
maksimum = concat['rata2 Lama akses materi'].max()
print("rata-rata kolom 1: ", rata_rata)
print("minimum kolom 1: ", minimum)
print("maksimum kolom 1: ", maksimum)

concat['lama_berlatih (min 5, mak 15)'] = concat['lama_berlatih (min 5, mak 15)'].apply(convert_to_minutes)
#rata rata kolom 2
rata_rata = concat['lama_berlatih (min 5, mak 15)'].mean()
#minimum kolom 2
minimum = concat['lama_berlatih (min 5, mak 15)'].min()
#maksimum kolom 2
maksimum = concat['lama_berlatih (min 5, mak 15)'].max()

print("rata-rata kolom 2: ", rata_rata)
print("minimum kolom 2: ", minimum)
print("maksimum kolom 2: ", maksimum)

#rata rata kolom 3
rata_rata = concat['nilai'].mean()
#minimum kolom 3
minimum = concat['nilai'].min()
#maksimum kolom 3
maksimum = concat['nilai'].max()

print("rata-rata kolom 3: ", rata_rata)
print("minimum kolom 3: ", minimum)
print("maksimum kolom 3: ", maksimum)

len(concat)

rata-rata kolom 1:  434.6355140186916
minimum kolom 1:  109
maksimum kolom 1:  690
rata-rata kolom 2:  15.405763239875387
minimum kolom 2:  2.0166666666666666
maksimum kolom 2:  119.48333333333333
rata-rata kolom 3:  76.86514018691588
minimum kolom 3:  0.0
maksimum kolom 3:  100.0


107